In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

import os
import time
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from matminer.featurizers.conversions import StrToComposition

In [ ]:
df_cp = StrToComposition().featurize_dataframe(df, "Formula",ignore_errors=False)

In [ ]:
from matminer.featurizers.composition import ElementProperty

In [ ]:
ep_magpie = ElementProperty.from_preset('magpie')
ep_deml = ElementProperty.from_preset('deml')
ep_matminer = ElementProperty.from_preset('matminer')
ep_matscholar = ElementProperty.from_preset('matscholar_el')
ep_megnet = ElementProperty.from_preset('megnet_el')
ep_feat = ElementProperty.from_preset(preset_name='magpie')

In [ ]:
df_out = ep_feat.featurize_dataframe(df_cp, col_id="composition",ignore_errors=True)

In [ ]:
ep_feat1 = ElementProperty.from_preset('deml')
df_out = ep_feat1.featurize_dataframe(df_out,col_id='composition',ignore_errors=False)

In [ ]:
ep_feat2 = ElementProperty.from_preset('matminer')
df_out = ep_feat2.featurize_dataframe(df_out,col_id='composition',ignore_errors=False)

In [ ]:
element_properties = pd.read_csv('C:/Users/HIT/Desktop/element_properties.csv',header = 0,index_col=0)
element_properties 

In [ ]:
total_properties_mp = []
total_pro = []
compound_pd = df_out['composition']
compound_np = np.array(compound_pd)

In [ ]:
add_features1 = [
                'atomic_volume','vdw_radius','density','lattice_constant',
    
               
                'evaporation_heat','specific_heat',
               
                
                'electron_affinity',
    
                
                'dipole_polarizability'
                ]
element_properties1 = element_properties[add_features1]

In [ ]:
def get_mean(compo_array,df_out):
    total_pro = []                           # 创建列表用于存储统计的性质
    # （1）提取composition和对应元素个数：
    for i in tqdm(range(len(compo_array))):
        compound_i = compound_np[i]           # 第i个composition
        compound_i_list = list(compound_i)    # 第i个composition列表（变成元素形式）
        elements = []                         # 建立储存元素的空集
        ele_num  = len(compound_i_list)       # 记录第i个composition元素的个数
        sum_ratio = 0                         # 初始化比例总和
        
    # （2）提取composition中的元素信息和占比：    
        for j in range(ele_num):               # 对第i个composition中，迭代每一个元素
            element_list = compound_i_list[j]  #  提取第i个composition中，第j个元素
            element_j = str(element_list)      #  将提取的第j个元素转化为字符型, 如"Al".
            ratio = compound_i[element_j]      #  提取第i个composition中，第j个元素所对应的原子占比数，如"Al"对应1.0
            sum_ratio += ratio                 #  将每一个j元素累加，形成总比例

    # （3）求化合物的统计性质："mean"                   
        mean_total = 0
        for j in range(ele_num):                       # 对第i个composition中，迭代每一个元素
            element_list = compound_i_list[j]           # 提取第i个composition中，第j个元素
            element_j = str(element_list)               # 将提取的第j个元素转化为字符型, 如"Al". 
            ratio = compound_i[element_j]
            s = element_properties1[element_properties1.symbol==element_j].index.tolist()# 提取第i个composition中，第j个元素所对应的原子占比数，如"Al"对应1.0
            ele_sym = element_properties2.loc[s]   # 将第j个元素在元素性质表中对应的行取出
            mean = ratio/sum_ratio*(np.array(ele_sym))   # 用第j个元素除以总比例形成权重，再与每一行的元素性质值相乘，得到j元素的加权性质
            mean_total += mean                            # 将每一个j元素加权的性质家和，形成这个composition的mean统计下的总属性

        total_pro.append(np.array(mean_total))
        
    df = total_pro[0].tolist()
    df = pd.DataFrame(df)                                    # 将每一个composition的mean统计性质存储到列表
    for i in range(1,len(total_pro)):
        data1 = pd.DataFrame(total_pro[i])
        data1 = pd.concat([df,data1],axis = 0 ,ignore_index = True)
        df = data1   
#     df = pd.DataFrame(total_pro,columns=add_features)   # 将统计列表转化为DF形式，并修改正确的列名
    df.columns = 'mean ' + element_properties2.columns                  # mean统计列名
    df_properties = pd.concat([df_out,df],axis=1)    # 将处理好的性质与原数据合并
     
    return df_properties   #返回处理好的df文件

In [ ]:
df_out1 = get_mean(compound_np,df_out)
df_out2 = get_std_dev(compound_np,df_out1)
df_out3 = get_entropy(compound_np,df_out2)
df_out4 = get_Gmean(compound_np,df_out3)
df_out5 = get_natoms(compound_np,df_out4)
df_out5.drop(['mean evaporation_heat','std_dev evaporation_heat','entropy evaporation_heat',
            'Gmean evaporation_heat','Gmean electron_affinity','composition'],axis = 1 ,inplace = True)
df_out5.to_excel('C:/Users/HIT/Desktop/12.xlsx',index=False)